Used car sales service is developing an application to attract new customers. There you can find the price of your car. 

We have historical data: technical characteristics, equipment and prices of cars. We need to build a model to determine the price.

We must check:

- score
- rmse
- time

# 1.About data

In [1]:
#library import
import pandas as pd
import lightgbm as lgbm
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error,mean_absolute_error
from lightgbm import LGBMRegressor
import timeit

In [2]:
#check data
autos = pd.read_csv('autos.csv')
display(autos.head())

,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,RegistrationMonth,FuelType,Brand,NotRepaired,DateCreated,NumberOfPictures,PostalCode,LastSeen
0,2016-03-24 11:52:17,480,NaN,1993,manual,0,golf,150000,0,petrol,volkswagen,NaN,2016-03-24 00:00:00,0,70435,2016-04-07 03:16:57
1,2016-03-24 10:58:45,18300,coupe,2011,manual,190,NaN,125000,5,gasoline,audi,yes,2016-03-24 00:00:00,0,66954,2016-04-07 01:46:50
2,2016-03-14 12:52:21,9800,suv,2004,auto,163,grand,125000,8,gasoline,jeep,NaN,2016-03-14 00:00:00,0,90480,2016-04-05 12:47:46
3,2016-03-17 16:54:04,1500,small,2001,manual,75,golf,150000,6,petrol,volkswagen,no,2016-03-17 00:00:00,0,91074,2016-03-17 17:40:17
4,2016-03-31 17:25:20,3600,small,2008,manual,69,fabia,90000,7,gasoline,skoda,no,2016-03-31 00:00:00,0,60437,2016-04-06 10:17:21


In [3]:
#str.lower
autos.columns = autos.columns.str.lower()
autos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 16 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   datecrawled        354369 non-null  object
 1   price              354369 non-null  int64 
 2   vehicletype        316879 non-null  object
 3   registrationyear   354369 non-null  int64 
 4   gearbox            334536 non-null  object
 5   power              354369 non-null  int64 
 6   model              334664 non-null  object
 7   kilometer          354369 non-null  int64 
 8   registrationmonth  354369 non-null  int64 
 9   fueltype           321474 non-null  object
 10  brand              354369 non-null  object
 11  notrepaired        283215 non-null  object
 12  datecreated        354369 non-null  object
 13  numberofpictures   354369 non-null  int64 
 14  postalcode         354369 non-null  int64 
 15  lastseen           354369 non-null  object
dtypes: int64(7), object(

In [4]:
autos.isnull().sum()

datecrawled              0
price                    0
vehicletype          37490
registrationyear         0
gearbox              19833
power                    0
model                19705
kilometer                0
registrationmonth        0
fueltype             32895
brand                    0
notrepaired          71154
datecreated              0
numberofpictures         0
postalcode               0
lastseen                 0
dtype: int64

In [5]:
#fill 0
autos['notrepaired'] = autos['notrepaired'].fillna(value='0')
autos.loc[autos['notrepaired'] == 'no', 'notrepaired'] = '0'
autos.loc[autos['notrepaired'] == 'yes', 'notrepaired'] = '1'

autos['gearbox'] = autos['gearbox'].fillna(value='0')
autos.loc[autos['gearbox'] == 'manual', 'gearbox'] = '0'
autos.loc[autos['gearbox'] == 'auto', 'gearbox'] = '1'

autos['fueltype'] = autos['fueltype'].fillna(value='no info')
autos['vehicletype'] = autos['vehicletype'].fillna(value='no info')
autos['model'] = autos['model'].fillna(value='no info')

autos.isnull().sum()

datecrawled          0
price                0
vehicletype          0
registrationyear     0
gearbox              0
power                0
model                0
kilometer            0
registrationmonth    0
fueltype             0
brand                0
notrepaired          0
datecreated          0
numberofpictures     0
postalcode           0
lastseen             0
dtype: int64

In [6]:
#change columns types
autos['notrepaired'] = autos['notrepaired'].astype('int')
autos['gearbox'] = autos['gearbox'].astype('int')
autos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 16 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   datecrawled        354369 non-null  object
 1   price              354369 non-null  int64 
 2   vehicletype        354369 non-null  object
 3   registrationyear   354369 non-null  int64 
 4   gearbox            354369 non-null  int32 
 5   power              354369 non-null  int64 
 6   model              354369 non-null  object
 7   kilometer          354369 non-null  int64 
 8   registrationmonth  354369 non-null  int64 
 9   fueltype           354369 non-null  object
 10  brand              354369 non-null  object
 11  notrepaired        354369 non-null  int32 
 12  datecreated        354369 non-null  object
 13  numberofpictures   354369 non-null  int64 
 14  postalcode         354369 non-null  int64 
 15  lastseen           354369 non-null  object
dtypes: int32(2), int64(7

In [7]:
#check duplicates
#autos.duplicated().sum()
autos = autos.drop_duplicates().reset_index(drop=True)
autos.duplicated().sum()

0

#### Conclusion

354364 entries и 16 columns:

Feauters:
- DateCrawled — Date crawled questionnaire from base
- VehicleType — Vehicle type
- RegistrationYear — Registration year
- Gearbox — Gearbox type
- Power — Power
- Model — car model
- Kilometer — mileage (km)
- RegistrationMonth — Registration month
- FuelType — Fuel type
- Brand — car brand
- NotRepaired — repair or not
- DateCreated — Date created questionnaire
- NumberOfPictures — Number of pictures
- PostalCode — Owners postal code
- LastSeen — Last seen of user

Target:
- Price — цена (euro)


We had 4 dulicates and now they have been deleted. 

In columns NotRepaired и Gearbox we change data for 1 и 0:

***NotRepaired:***
- no = 0
- yes = 1

***Gearbox:***
- manual = 0
- auto = 1

Also, nulls in FuelType, VehicleType и Model now is "no info".

# 2. Train and test models

In [8]:
#we should delete this columns, because we don't need them in model and haven't enough memory
pre_auto = autos.drop(['datecrawled', 'registrationmonth','datecreated',
                      'numberofpictures', 'postalcode','lastseen'], axis=1)

In [9]:
#get dummies
auto = pd.get_dummies(pre_auto, drop_first=True)
auto.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354364 entries, 0 to 354363
Columns: 310 entries, price to brand_volvo
dtypes: int32(2), int64(4), uint8(304)
memory usage: 116.3 MB


In [10]:
#feauters & target
features = auto.drop(['price'], axis=1)
target = auto['price']

#divide for two
features_train_valid, features_test, target_train_valid, target_test = train_test_split(
    features, target, test_size=0.20, random_state=12345)

#and three
features_train, features_valid, target_train, target_valid = train_test_split(
    features_train_valid, target_train_valid, test_size=0.250, random_state=12345)

print(features_train.shape)
print(features_valid.shape)
print(features_test.shape)

(212618, 309)
(70873, 309)
(70873, 309)


In [11]:
#depth params
for depth in range(5, 11):
    model =  DecisionTreeRegressor(random_state=12345, max_depth=depth)
    model.fit(features_train, target_train)
    predictions_valid = model.predict(features_valid)
    rmse = (mean_squared_error(target_valid,predictions_valid))**0.5
    print("max_depth =", depth, ": ", end='') 
    print('RMSE:', rmse)

max_depth = 5 : RMSE: 2560.224586970118
max_depth = 6 : RMSE: 2433.1661132195786
max_depth = 7 : RMSE: 2320.172601434079
max_depth = 8 : RMSE: 2253.4671623423774
max_depth = 9 : RMSE: 2194.908138915063
max_depth = 10 : RMSE: 2150.795417687194


In [12]:
#params n_estimators
for est in range(5, 11):
    model =  RandomForestRegressor(random_state=12345, n_estimators = est)
    model.fit(features_train, target_train)
    predictions_valid = model.predict(features_valid)
    rmse = (mean_squared_error(target_valid,predictions_valid))**0.5
    print("max_est =", est, ": ", end='')
    print('RMSE:', rmse)

max_est = 5 : RMSE: 1867.7239487034965
max_est = 6 : RMSE: 1856.0096237518605
max_est = 7 : RMSE: 1844.0437504300783
max_est = 8 : RMSE: 1833.5423935286428
max_est = 9 : RMSE: 1824.6751123281801
max_est = 10 : RMSE: 1818.1471155615225


In [13]:
#train DecisionTreeRegressor
model =  DecisionTreeRegressor(random_state=12345, max_depth=6)
model.fit(features_train, target_train)
predictions_valid = model.predict(features_valid)
rmse = (mean_squared_error(target_valid,predictions_valid))**0.5
print(model.score(features_valid, target_valid))
print('RMSE:', rmse)

0.7097136203726361
RMSE: 2433.1661132195786


In [14]:
#train RandomForestRegressor
model =  RandomForestRegressor(random_state=12345, n_estimators = 6)
model.fit(features_train, target_train)
predictions_valid = model.predict(features_valid)
rmse = (mean_squared_error(target_valid,predictions_valid))**0.5
print(model.score(features_valid, target_valid))
print('RMSE:', rmse)

0.8310945792510871
RMSE: 1856.0096237518605


#### Conclusion

RMSE is normal(for our price).
Score sometimes(RandomForestRegressor) is high, but all is normal.

In [15]:
#test RandomForestRegressor
model = RandomForestRegressor(random_state=12345, n_estimators = 6)
model.fit(features_train, target_train)
score_forest = model.score(features_test, target_test)

predictions_test = model.predict(features_test)
rmse_forest = (mean_squared_error(target_test,predictions_test))**0.5
time_forest = timeit.timeit()

print('score:', score_forest)
print('RMSE:', rmse_forest)
print('time:',time_forest)

score: 0.8298396647684432
RMSE: 1860.4281849360746
time: 0.006172300000002906


In [16]:
#test DecisionTreeRegressor
model = DecisionTreeRegressor(random_state=12345, max_depth=6)
model.fit(features_train, target_train)
score_tree = model.score(features_test, target_test)

predictions_test = model.predict(features_test)
rmse_tree = (mean_squared_error(target_test,predictions_test))**0.5
time_tree = timeit.timeit()
print('score:',score_tree)
print('RMSE:', rmse_tree)
print('time:',time_tree)

score: 0.7125152539014175
RMSE: 2418.194043560093
time: 0.009367600000018683


In [17]:
#category for lgbm
auto_lgbm = pre_auto.copy(deep=True)
auto_lgbm[['model','fueltype','brand','vehicletype']] = auto_lgbm[['model','fueltype','brand','vehicletype']].astype('category')

display(auto_lgbm.head())

,price,vehicletype,registrationyear,gearbox,power,model,kilometer,fueltype,brand,notrepaired
0,480,no info,1993,0,0,golf,150000,petrol,volkswagen,0
1,18300,coupe,2011,0,190,no info,125000,gasoline,audi,1
2,9800,suv,2004,1,163,grand,125000,gasoline,jeep,0
3,1500,small,2001,0,75,golf,150000,petrol,volkswagen,0
4,3600,small,2008,0,69,fabia,90000,gasoline,skoda,0


In [18]:
#feauters and target for lgbm
features_lgbm = auto_lgbm.drop(['price'], axis=1)
target_lgbm = auto_lgbm['price']

#divide for two
features_train_valid_lgbm, features_test_lgbm, target_train_valid_lgbm, target_test_lgbm = train_test_split(
    features_lgbm, target_lgbm, test_size=0.20, random_state=12345)

#and three
features_train_lgbm, features_valid_lgbm, target_train_lgbm, target_valid_lgbm = train_test_split(
    features_train_valid_lgbm, target_train_valid_lgbm, test_size=0.250, random_state=12345)

print(features_train_lgbm.shape)
print(features_valid_lgbm.shape)
print(features_test_lgbm.shape)

(212618, 9)
(70873, 9)
(70873, 9)


In [19]:
#params for lgbm
lgbm.LGBMRegressor().fit(features_train_lgbm,target_train_lgbm)


LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
              importance_type='split', learning_rate=0.1, max_depth=-1,
              min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
              n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
              random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
              subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [20]:
#params for LightGBM max_depth
for depth in range(5, 11):
    model =  LGBMRegressor(random_state=12345, max_depth=depth)
    model.fit(features_train_lgbm,target_train_lgbm)
    predictions_valid = model.predict(features_valid_lgbm)
    rmse = (mean_squared_error(target_valid_lgbm,predictions_valid))**0.5
    print("max_depth =", depth, ": ", end='') 
    print('RMSE:', rmse)

max_depth = 5 : RMSE: 1832.069376652848
max_depth = 6 : RMSE: 1821.748200812587
max_depth = 7 : RMSE: 1809.3383827281925
max_depth = 8 : RMSE: 1803.4048773882464
max_depth = 9 : RMSE: 1802.5455865738786
max_depth = 10 : RMSE: 1798.7490362984374


In [21]:
#params for  LightGBM num_leaves
for nl in range(5, 11):
    model =  LGBMRegressor(random_state=12345, num_leaves=nl)
    model.fit(features_train_lgbm,target_train_lgbm)
    predictions_valid = model.predict(features_valid_lgbm)
    rmse = (mean_squared_error(target_valid_lgbm,predictions_valid))**0.5
    print("num_leaves =", nl, ": ", end='') 
    print('RMSE:', rmse)

num_leaves = 5 : RMSE: 1993.947161223378
num_leaves = 6 : RMSE: 1961.570239386642
num_leaves = 7 : RMSE: 1942.5030816982214
num_leaves = 8 : RMSE: 1924.0543177790512
num_leaves = 9 : RMSE: 1904.38201761778
num_leaves = 10 : RMSE: 1895.3654661180635


In [22]:
#params for  LightGBM min_data_in_leaf
for min_data in range(5, 11):
    model = LGBMRegressor(random_state=12345, min_data_in_leaf=min_data)
    model.fit(features_train_lgbm,target_train_lgbm)
    predictions_valid = model.predict(features_valid_lgbm)
    rmse = (mean_squared_error(target_valid_lgbm,predictions_valid))**0.5
    print("min_data_in_leaf =", min_data, ": ", end='') 
    print('RMSE:', rmse)

min_data_in_leaf = 5 : RMSE: 1799.866154751988
min_data_in_leaf = 6 : RMSE: 1802.5689166299237
min_data_in_leaf = 7 : RMSE: 1800.6876956068552
min_data_in_leaf = 8 : RMSE: 1802.1500000568453
min_data_in_leaf = 9 : RMSE: 1801.8822078391745
min_data_in_leaf = 10 : RMSE: 1800.8330339716686


In [23]:
#train LGBMRegressor
model = LGBMRegressor(random_state=12345,max_depth=6, max_bin=6, min_data_in_leaf = 7,num_leaves=6 )
model.fit(features_train_lgbm,target_train_lgbm)
predictions_valid_lgbm = model.predict(features_valid_lgbm)
rmse_lgbm = (mean_squared_error(target_valid_lgbm,predictions_valid_lgbm))**0.5
print(model.score(features_valid_lgbm, target_valid_lgbm))
print('RMSE:', rmse_lgbm)

0.7735769721132164
RMSE: 2148.9124735639084


In [24]:
#test LGBMRegressor
model = LGBMRegressor(random_state=12345,max_depth=6, max_bin=6, min_data_in_leaf = 7,num_leaves=6 )
model.fit(features_train_lgbm, target_train_lgbm)
score_lgbmr = model.score(features_test_lgbm, target_test_lgbm)

predictions_test_lgbm = model.predict(features_test_lgbm)
rmse_lgbmr = (mean_squared_error(target_test_lgbm,predictions_test_lgbm))**0.5
time_lgbmr = timeit.timeit()
print('score:',score_lgbmr)
print('RMSE:', rmse_lgbmr)
print('time:',time_lgbmr)

score: 0.7742535929985919
RMSE: 2142.8618224695506
time: 0.015763699999979508


# 3. Analyze

In [25]:
#table
rmse_score_time = pd.DataFrame({'regression':['RandomForestRegressor','DecisionTreeRegressor',
                                              'LGBMRegressor'],
                                'rmse':[rmse_forest, rmse_tree, rmse_lgbmr], 
                                'score':[score_forest,score_tree,score_lgbmr],
                                'time':[time_forest,time_tree,time_lgbmr]})

display(rmse_score_time)

,regression,rmse,score,time
0,RandomForestRegressor,1860.428185,0.829840,0.006172
1,DecisionTreeRegressor,2418.194044,0.712515,0.009368
2,LGBMRegressor,2142.861822,0.774254,0.015764


### Final conclusion 

Score is real, it is very good. Time is too small and it is good too.

Lowest rmse has RandomForestRegressor,but it has highest score: 1860 и 0.83

DecisionTreeRegressor has highest rmse(24018) and smallest score - 0.71.

LGBMRegressor in a middle: score - 0,77, rmse - 2143. 

So, here we can choose, what we need: better rmse or better score.

But company can choose, what we need: better rmse or better score.